## Importing Modules

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as opt
from torch.utils.data import DataLoader, random_split, TensorDataset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Taking a look at the Dataset

In [7]:
df = pd.read_csv("global_temp.csv")

print(df.head(1))

           dt  ...  LandAndOceanAverageTemperatureUncertainty
0  1750-01-01  ...                                        NaN

[1 rows x 9 columns]


## Fixing the data and creating the input and output tensor

In [8]:
numpy_dataset = np.genfromtxt("global_temp.csv", delimiter=',')

for i in range(len(df["dt"])):
    data_to_float = '.'.join(df["dt"][i].split('-')[:-1])
    numpy_dataset[i + 1][0] = data_to_float

list_dataset_specific = []
for rec in numpy_dataset[1:]:
    if np.isnan(rec[1]):
        rec[1] = 0
    list_dataset_specific.append([rec[0], rec[1]])

list_dataset_fixed = []
temp = []
j = 1753
for i in range(len(list_dataset_specific[36:])):
    if (i % 12 == 0) and (i != 0):
        list_dataset_fixed.append([j] + temp)
        j += 1
        temp = []

    temp.append(list_dataset_specific[36:][i][1])

list_dataset_fixed.append(temp)

inputs = []
outputs = []
for index in range(len(list_dataset_fixed)):
    inputs.append(list_dataset_fixed[index][0])
    outputs.append(np.average(list_dataset_fixed[index][2:]))

inputs_tensor = torch.tensor(inputs, dtype=torch.float32)
outputs_tensor = torch.tensor(outputs, dtype=torch.float32)


## Hyperparameters

In [9]:
batch_size = 32
train_size = 173
valid_size = 70
test_size = 20

input_size = 1
output_size = 1

h1_size = 10
h2_size = 32
h3_size = 64
h4_size = 128
h5_size = 100
h6_size = 50
h7_size = 10

## Creating the Dataloaders

In [10]:
train_ds, valid_ds, test_ds = random_split(TensorDataset(inputs_tensor, outputs_tensor), [train_size, valid_size, test_size])

train_dl = DataLoader(train_ds, batch_size, num_workers=2, pin_memory=True, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size, num_workers=2, pin_memory=True)

## Moving tensors to Device (cuda or cpu)

In [11]:
class DeviceLoader:
    def __init__(self, device, loader):
        self.device = device
        self.loader = loader

    def __to_device(self, data):
        if isinstance(data, (list, tuple)):
            on_device_batch = [d.to(self.device, non_blocking=True) for d in data]
            return on_device_batch
        return data.to(self.device, non_blocking=True)

    def __iter__(self):
        for batch in self.loader:
            yield self.__to_device(batch)

    def __len__(self):
        return len(self.loader)

## Creating the Model

In [102]:
class TemperatureModel(nn.Module):
    def __init__(self, in_size, h1_size, h2_size, h3_size, h4_size, h5_size, h6_size, h7_size, out_size):
        super().__init__()
        self.linear1 = nn.Linear(in_size, h1_size)
        self.linear2 = nn.Linear(h1_size, h2_size)
        self.linear3 = nn.Linear(h2_size, h3_size)
        self.linear4 = nn.Linear(h3_size, h4_size)
        self.linear5 = nn.Linear(h4_size, h5_size)
        self.linear6 = nn.Linear(h5_size, h6_size)
        self.linear7 = nn.Linear(h6_size, h7_size)
        self.linear8 = nn.Linear(h7_size, out_size)

    def __call__(self, input_batch):
        out1 = self.linear1(input_batch)
        activation_out1 = F.relu(out1)
        out2 = self.linear2(activation_out1)
        activation_out2 = F.relu(out2)
        out3 = self.linear3(activation_out2)
        activation_out3 = F.relu(out3)
        out4 = self.linear4(activation_out3)
        activation_out4 = F.relu(out4)
        out5 = self.linear5(activation_out4)
        activation_out5 = F.relu(out5)
        out6 = self.linear6(activation_out5)
        activation_out6 = F.relu(out6)
        out7 = self.linear7(activation_out6)
        activation_out7 = F.relu(out7)
        model_pred = self.linear8(activation_out7)
        return model_pred

    def __validation_step(self, validation_batch):
        input_batch, output_batch = validation_batch
        model_preds = self(input_batch)
        loss = F.mse_loss(model_preds, output_batch)
        acc = 100 - (torch.mean(torch.abs((output_batch - model_preds) / output_batch)) * 100)
        return {"valid_batch_loss": torch.sqrt(loss).item(), "valid_batch_acc": acc}

    def __validation_end(self, batch_results):
        avg_loss = torch.tensor([x["valid_batch_loss"] for x in batch_results]).mean().item()
        avg_acc = torch.tensor([x["valid_batch_acc"] for x in batch_results]).mean().item()
        return {"valid_loss": avg_loss, "valid_acc": f"{avg_acc}%"}
    
    def training_step(self, training_batch):
        input_batch, output_batch = training_batch
        model_preds = self(input_batch)
        loss = F.mse_loss(model_preds, output_batch)
        return loss

    def evaluate(self, validation_loader):
        batch_results = [self.__validation_step(valid_batch) for valid_batch in validation_loader]
        return self.__validation_end(batch_results)

    def epoch_end(self, epoch, results):
        return {"Epoch: ": epoch+1, "Loss: ": results["valid_loss"], "Acc:": results["valid_acc"]}

    def predict(self, test_batch):
        input_test, output_test = test_batch
        preds = self(input_test.unsqueeze(dim=0))
        loss = F.mse_loss(preds, output_test)
        acc = 100 - torch.abs((output_test - preds) / output_test).item() * 100
        return {"Loss": loss.sqrt().item(), "Acc(%)": acc, "Actual": output_test.item(), "Predicted": preds.item()}

## Creating the model, the Device loaders and moving all the tensors to device

In [103]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

model = TemperatureModel(input_size, h1_size, h2_size, h3_size, h4_size, h5_size, h6_size, h7_size, output_size).to(device)

train_loader = DeviceLoader(device, train_dl)
valid_loader = DeviceLoader(device, valid_dl)
test_loader = DeviceLoader(device, test_ds)

## Creating the training loop

In [104]:
def fit(model, epochs, train_loader, valid_loader, opt_fn=opt.SGD, lr=1e-4):
    history = []
    optim = opt_fn(model.parameters(), lr=lr)
    for epoch in range(epochs):
        for training_batch in train_loader:
            loss = model.training_step(training_batch)
            loss.backward()
            optim.step()
            optim.zero_grad()

        valid_results = model.evaluate(valid_loader)
        epoch_results = model.epoch_end(epoch, valid_results)
        history.append(epoch_results)
        print(epoch_results)

    return history

## Training the Model

In [105]:
history = fit(model, 10, train_loader, valid_loader)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:46: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:46: UserWarning: Using a target size (torch.Size([13])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.


{'Epoch: ': 1, 'Loss: ': 0.5886731147766113, 'Acc:': '94.85871124267578%'}


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: UserWarning: Using a target size (torch.Size([6])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.


{'Epoch: ': 2, 'Loss: ': 1.037718653678894, 'Acc:': '89.76904296875%'}
{'Epoch: ': 3, 'Loss: ': 1.8443384170532227, 'Acc:': '80.98458099365234%'}
{'Epoch: ': 4, 'Loss: ': 0.6658509373664856, 'Acc:': '93.92556762695312%'}
{'Epoch: ': 5, 'Loss: ': 0.6594939231872559, 'Acc:': '94.27777099609375%'}
{'Epoch: ': 6, 'Loss: ': 0.8728339672088623, 'Acc:': '91.70928955078125%'}
{'Epoch: ': 7, 'Loss: ': 0.8010405898094177, 'Acc:': '92.46492767333984%'}
{'Epoch: ': 8, 'Loss: ': 0.7243301868438721, 'Acc:': '93.3253173828125%'}
{'Epoch: ': 9, 'Loss: ': 0.722100555896759, 'Acc:': '93.58057403564453%'}
{'Epoch: ': 10, 'Loss: ': 0.5895013213157654, 'Acc:': '94.8472900390625%'}


In [106]:
for test_batch in test_loader:
    print(model.predict(test_batch))

{'Loss': 0.16384410858154297, 'Acc(%)': 98.29337131232023, 'Actual': 9.600454330444336, 'Predicted': 9.764298439025879}
{'Loss': 0.31287097930908203, 'Acc(%)': 96.56340219080448, 'Actual': 9.104090690612793, 'Predicted': 9.416961669921875}
{'Loss': 0.43018627166748047, 'Acc(%)': 95.3815646469593, 'Actual': 9.314545631408691, 'Predicted': 9.744731903076172}
{'Loss': 0.6377496719360352, 'Acc(%)': 92.69906803965569, 'Actual': 8.73518180847168, 'Predicted': 9.372931480407715}
{'Loss': 0.5851249694824219, 'Acc(%)': 93.31931248307228, 'Actual': 8.758454322814941, 'Predicted': 9.343579292297363}
{'Loss': 0.3897533416748047, 'Acc(%)': 95.65263278782368, 'Actual': 8.965272903442383, 'Predicted': 8.575519561767578}
{'Loss': 1.2721834182739258, 'Acc(%)': 87.10703998804092, 'Actual': 9.86727237701416, 'Predicted': 8.595088958740234}
{'Loss': 0.14517784118652344, 'Acc(%)': 98.34361262619495, 'Actual': 8.764727592468262, 'Predicted': 8.619549751281738}
{'Loss': 0.6122369766235352, 'Acc(%)': 92.94058

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:59: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
